In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.utils import shuffle
from keras.utils import np_utils #One-hot-encoding
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


Using TensorFlow backend.
C:\Users\canni\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\canni\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\canni\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\canni\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
def generate_accuracy_and_heatmap(model, X, y):
#     cm = confusion_matrix(y,model.predict(X))
#     sns.heatmap(cm,annot=True,fmt="d")
    ac = accuracy_score(y,model.predict_classes(X))
    print('Accuracy is: ', ac)
    print (pd.crosstab(y,model.predict_classes(X),rownames=['label'],colnames=['predict']))
    ##metrics.confusion_matrix(y_test, clf_lr.predict(X_test_selected)) 
    return 1


In [3]:
#%% import data
os.chdir('D:/data2')
path = "D:/data2" #資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱


In [4]:
df = pd.DataFrame() #空的dataframe
for i in range(81):
    d2=pd.read_csv(files[i])
    if d2.isnull().sum().sum() != 0:
        print("In ",files[i])
        for j in range(len(d2.index)) :
            if d2.iloc[j].isnull().sum() != 0:
                print("Nan in row ", j+1 , " : " ,  d2.iloc[j].isnull().sum())
        print(d2)
    df=pd.concat([df,d2],axis=0)  
df = df.reset_index()
X=df.drop(['Unnamed: 0','V115', 'V116','V117','index'], axis=1)



In  Assay 20.csv
Nan in row  43  :  114
    Unnamed: 0        V1        V2        V3        V4        V5        V6  \
0            1  1.704955  0.755850  1.076499  1.640432  2.844560  3.328134   
1            2  1.484763  0.738368  1.040285  1.505723  2.686339  3.286160   
2            3  1.662222  0.726194  0.993292  1.616492  2.710324  3.186518   
3            4  1.543112  0.690364  0.963098  1.428123  2.351463  2.788580   
4            5  1.752083  0.739432  1.014502  1.451558  2.496606  3.039571   
5            6  1.578395  0.693596  0.931738  1.422942  2.207460  2.895355   
6            7  1.593588  0.740670  0.976849  1.565033  2.427810  2.837873   
7            8  1.474497  0.682559  0.916265  1.409606  2.174837  2.769269   
8            9  1.962298  0.973986  1.400845  1.858015  2.423164  2.914568   
9           10  2.150923  0.978806  1.483732  2.004222  2.592350  3.149516   
10          11  1.897247  0.857523  1.272912  1.745464  2.440882  2.839146   
11          12  1.820878

In [5]:

A=np.repeat(0,16)
B=np.repeat(1,8)
C=np.repeat(0,23)
yf=np.hstack((A,B,C))
yf=np.tile(yf,81)
print('Train:',X.shape,yf.shape)
y_num=[(yf==i).sum() for i in range(6)]
print("total number of each class: ",y_num)

Train: (3807, 114) (3807,)
total number of each class:  [3159, 648, 0, 0, 0, 0]


In [6]:
#%% delete missing data
def missing(X):
    miss=[]
    for i in range(len(X.index)) :
        if X.iloc[i].isnull().sum() != 0:
            print("Nan in row ", i , " : " ,  X.iloc[i].isnull().sum())
            miss.append(i)
    return miss
miss=missing(X)
X=X.drop(miss)
yf=np.delete(yf, miss)
print(X.shape,yf.shape)

Nan in row  606  :  114
Nan in row  685  :  114
(3805, 114) (3805,)


In [7]:
#%% Separate data to to training,validation,testing data
### Before applying feature selection method, we need to split the data first.
acc_train=[]
acc_test=[]
sfolder = StratifiedKFold(n_splits=10,random_state=0,shuffle=True)

for train_index, test_index in sfolder.split(X,yf):
    print("===========================================================================")
    #print('Train: %s \nTest: %s' % (train_index, test_index))
    X_train, X_test =X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = np.array(yf)[train_index], np.array(yf)[test_index]
    print("Number of training data:",y_train.shape[0],"\nNumber of testing data:",y_test.shape[0])
    print([(y_train==i).sum() for i in range(6)])
    print([(y_test==i).sum() for i in range(6)])
    print(" ")
    X_train, y_train = shuffle(X_train, y_train, random_state=2)


Number of training data: 3424 
Number of testing data: 381
[2841, 583, 0, 0, 0, 0]
[316, 65, 0, 0, 0, 0]
 
Number of training data: 3424 
Number of testing data: 381
[2841, 583, 0, 0, 0, 0]
[316, 65, 0, 0, 0, 0]
 
Number of training data: 3424 
Number of testing data: 381
[2841, 583, 0, 0, 0, 0]
[316, 65, 0, 0, 0, 0]
 
Number of training data: 3424 
Number of testing data: 381
[2841, 583, 0, 0, 0, 0]
[316, 65, 0, 0, 0, 0]
 
Number of training data: 3424 
Number of testing data: 381
[2841, 583, 0, 0, 0, 0]
[316, 65, 0, 0, 0, 0]
 
Number of training data: 3425 
Number of testing data: 380
[2841, 584, 0, 0, 0, 0]
[316, 64, 0, 0, 0, 0]
 
Number of training data: 3425 
Number of testing data: 380
[2841, 584, 0, 0, 0, 0]
[316, 64, 0, 0, 0, 0]
 
Number of training data: 3425 
Number of testing data: 380
[2842, 583, 0, 0, 0, 0]
[315, 65, 0, 0, 0, 0]
 
Number of training data: 3425 
Number of testing data: 380
[2842, 583, 0, 0, 0, 0]
[315, 65, 0, 0, 0, 0]
 
Number of training data: 3425 
Number

In [8]:
 ### One-hot-encoding
num_classes = 6
Onehot_train = np_utils.to_categorical(y_train, num_classes)
Onehot_test = np_utils.to_categorical(y_test, num_classes)
    ### model
n=X_train.shape[0]
p=X_train.shape[1]
model = Sequential()
model.add(Dense(units=int(p*2/3+num_classes),input_dim=p,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
### Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Onehot_train, epochs=50, batch_size=25,verbose=0)
acc_train.append(round(accuracy_score(y_train,model.predict_classes(X_train)),4))
acc_test.append(round(accuracy_score(y_test,model.predict_classes(X_test)),4))


In [9]:
print("Confusion matrix of FGFR1 for training data:")
generate_accuracy_and_heatmap(model, X_train, y_train)
print("Confusion matrix of FGFR1 for testing data:")
generate_accuracy_and_heatmap(model, X_test, y_test)

from statistics import mean
print("ACC_train:",acc_train," mean:",mean(acc_train))
print("ACC_test:",acc_test," mean:",mean(acc_test))
    

Confusion matrix of FGFR1 for training data:
Accuracy is:  0.9617518248175182
predict     0    1
label             
0        2807   35
1          96  487
Confusion matrix of FGFR1 for testing data:
Accuracy is:  0.9578947368421052
predict    0   1
label           
0        309   6
1         10  55
ACC_train: [0.9618]  mean: 0.9618
ACC_test: [0.9579]  mean: 0.9579
